# QGAN

Source: https://github.com/jgalfaro/mirrored-QGANMAV

In [4]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import GradientDescentOptimizer
import math

In [77]:
# load data
np.random.seed(0)
real_values = np.random.normal(size=(2**num_qubits,))
# Normalization function
def normalize(values):
    mean = np.sqrt(np.sum(values ** 2))
    return  np.array(values / mean)
real_values = normalize(real_values)
assert np.abs(1-np.sum(real_values ** 2))<=0.0000001
print("Normalized data: ",real_values)

Normalized data:  [ 0.2079372   0.04716842  0.11536848  0.26414469  0.2201379  -0.11519637
  0.11199142 -0.01784119 -0.01216689  0.04839919  0.0169791   0.1714221
  0.08970712  0.01434241  0.05232026  0.03933177  0.17611417 -0.02418298
  0.03690277 -0.10067631 -0.300933    0.07704512  0.10189519 -0.0874825
  0.26754672 -0.17143297  0.00539377 -0.02206425  0.18067594  0.17320027
  0.01826439  0.04457581 -0.10464751 -0.23348586 -0.04101005  0.01842959
  0.14502019  0.1417302  -0.04565605 -0.03563386 -0.1235979  -0.16738424
 -0.20112614  0.22994713 -0.06007512 -0.05163789 -0.14767292  0.09164647
 -0.19023778 -0.0250767  -0.10555288  0.04560603 -0.06021102 -0.1391667
 -0.00332197  0.05048951  0.0078407   0.03565379 -0.07477055 -0.04275802
 -0.0792661  -0.04238223 -0.0958494  -0.2034851 ]


### Set up Discriminator

In [78]:
num_qubits = 6
dev = qml.device('default.qubit', wires=num_qubits)

In [59]:
def layer(num_wires,W):
    for i in range(num_wires):
       qml.Rot(W[i, 0], W[i, 1], W[i, 2], wires=i)
    for i in range(num_wires):
        qml.CNOT(wires=[i, (i+1)%num_wires])

In [60]:
@qml.qnode(dev)
def real_disc_circuit(real_values,weights):
    # initialize the input state real data
    qml.QubitStateVector(real_values,wires=range(num_qubits))
    # assign weights to layers
    for W in weights:
        layer(num_qubits,W)
    # measure
    return qml.expval(qml.PauliZ(0))

In [61]:
@qml.qnode(dev)
def gen_disc_circuit(fake_values,disc_weights):
    # initialize with generator data
    qml.QubitStateVector(fake_values, wires=range(num_qubits))
    # discriminator layers
    for W in disc_weights:
        layer(W)
    # measure
    return qml.expval(qml.PauliZ(0))

In [62]:
np.random.seed(0)
num_layers = 2
# discriminator weights
disc_weights = 0.01 * np.random.randn(num_layers, num_qubits, 3)
# test the discriminator circuit
r = real_disc_circuit(real_values, disc_weights)
assert(r>=-1 and r<=1)
print("Discriminator expectation (test mode): ", r)

Discriminator expectation (test mode):  0.09117979832534029


In [63]:
def prob_real_true(real_values,disc_weights):
    # get measurement
    r = real_disc_circuit(real_values,disc_weights)
    assert(r>=-1 and r<=1)
    # convert "r" to a probability
    p = (r + 1) / 2
    assert(p>=0 and r<=1)
    return p

In [ ]:
def prob_fake_true(fake_values,disc_weights):
    # get measurement
    r = real_disc_circuit(fake_values,disc_weights)
    assert(r>=-1 and r<=1)
    # convert "r" to a probability
    p = (r + 1) / 2
    assert(p>=0 and r<=1)
    return p

In [64]:
def disc_cost(real_values,disc_weights):
    cost = - prob_real_true(real_values,disc_weights)
    return cost

In [65]:
# random fake values
fake_values =  0.01 * np.random.randn(2**num_qubits)
# normalized fake values
norm_fake_values = normalize(fake_values)
# check consistency of probability amplitudes
assert np.abs(1-np.sum(norm_fake_values ** 2))<=0.0000001
print("Normalized fake data: ",norm_fake_values)
# test the generator circuit
#r = gen_disc_circuit(norm_fake_values,disc_weights)
#assert(r>=-1 and r<=1)
#print("Generator expectation (test mode): ", r)

Normalized fake data:  [ 0.16451982  0.16078746 -0.05179503 -0.04042524 -0.14021706 -0.18989097
 -0.22816986  0.26086616 -0.0681529  -0.0585812  -0.16752924  0.10396939
 -0.21581743 -0.02844855 -0.11974568  0.05173828 -0.06830708 -0.15787927
 -0.00376865  0.0572784   0.00889497  0.04044786 -0.08482431 -0.04850733
 -0.08992433 -0.04808101 -0.10873745 -0.23084601  0.02372619 -0.05372789
 -0.21799721  0.06188525 -0.12132788  0.00694636  0.09749716  0.01724816
  0.15236561 -0.16512627  0.05380287 -0.09157578 -0.11644678 -0.07740629
 -0.04166216  0.00751067 -0.1558089   0.12046243  0.06227041 -0.20543318
  0.19901555  0.25352654  0.15763153 -0.02406033 -0.14318569  0.14100587
 -0.05391457  0.16347067  0.02785144  0.13060042  0.04765486  0.09448603
  0.00140411  0.23881436  0.01697124  0.05375576]


In [79]:
# create the optimizer
opt = GradientDescentOptimizer(0.1)
for i in range(50):
    j = 0
    disc_weights = opt.step(lambda v: disc_cost(real_values,v),disc_weights)
    cost = disc_cost(real_values, disc_weights)
    if i % 5 == 0:
        print("Step {}: cost = {}".format(i+1, cost))
p_R = prob_real_true(real_values, disc_weights)
assert(p_R>=0 and p_R<=1)
print("Probability of real true: ", p_R)

Step 1: cost = -0.7642873839361863
Step 6: cost = -0.7728274363257351
Step 11: cost = -0.7803572633152197
Step 16: cost = -0.7870965106122925
Step 21: cost = -0.7932036093840127
Step 26: cost = -0.7987925294268955
Step 31: cost = -0.8039453673401826
Step 36: cost = -0.8087211933334987
Step 41: cost = -0.8131622214737488
Step 46: cost = -0.8172982744500491
Probability of real true:  0.8204017775341923


## Set up Generator

In [69]:
dev_gaussian = qml.device('default.gaussian', wires=2**num_qubits)

In [70]:
@qml.qnode(dev_gaussian)
def mean_photon_gaussian(params):
    for i in range(2**num_qubits):
        qml.Displacement(params[i][0],params[i][1], wires=i)
        qml.Rotation(params[i][2], wires=i)
    return [qml.expval(qml.NumberOperator(i)) for i in range(2**num_qubits)]

In [71]:
init_params = 0.1*np.ones([2**num_qubits,3],dtype=float)
mean_photon_gaussian(init_params)

tensor([0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
        0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
        0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
        0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
        0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
        0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01], requires_grad=True)

In [72]:
def gen_cost(params,disc_weights):
    # calculate expected number of photons
    fake_values = mean_photon_gaussian(params)
    # normalize fake values
    norm_fake_values = normalize(fake_values)
    assert np.abs(1-np.sum(norm_fake_values ** 2))<=0.0000001
    # determine the probability of recognizing them as true values
    cost = - prob_real_true(norm_fake_values,disc_weights)
    return cost

In [73]:
# generator weights
gen_weights = 0.1*np.ones([2**num_qubits,3],dtype=float)
gen_cost(gen_weights,disc_weights)

tensor(-0.50495621, requires_grad=True)

In [81]:
# initialize the optimizer
opt = qml.GradientDescentOptimizer(stepsize=0.1)
# set the number of steps
steps = 20
print("...")
for i in range(steps):
    # update the circuit parameters
    gen_weights = opt.step(lambda v: gen_cost(v,disc_weights),gen_weights)
    cost = gen_cost(gen_weights,disc_weights)
    if i % 5 == 0:
        print("Step {}: cost = {}".format(i+1, cost))

...


TypeError: loop of ufunc does not support argument 0 of type ArrayBox which has no callable sqrt method